# EEG Preprocessing

## Make EEG Preprocessing Derivatives File Structure

EEG isn't standardly supported by the BIDS specification yet. However, we still try to stay reasonably close to the derivatives philosophy currently under development. Thus the preprocessing section (which culminates in cleaned epochs) is stored as an eeg_preprocessing BIDS-derivatives compatible pipeline in the data/derivatives/eeg_preprocessing folder.

In [11]:
make_eeg_prep_derivatives_folder('../data')

ImportError: No module named src

## Mark Bad Channels


In [1]:
from grabbit import Layout

%matplotlib

# extract participant raw filenames
layout = Layout('../data', '../data/grabbit_config.json')
raw_files = layout.get(subject='sub-.*', modality='eeg', 
                       extensions='.fif')

# iterate through participant raw data
for f in raw_files:
    
    manually_mark_bad_channels(f)


Using matplotlib backend: Qt4Agg


## Generate Epochs

### Verify Cleaned Epochs

In [2]:
from msit import visually_verify_epochs

In [3]:
visually_verify_epochs?

In [1]:
import json
from grabbit import Layout
from msit import load_epochs, visually_verify_epochs
import numpy as np

%matplotlib

set_log_level('critical')

layout = Layout('../data', '../data/grabbit_config.json')
subjects = np.array(sorted(layout.get(target='subject', 
                                      modality='eeg',
                                      return_type='id')))
pipeline_root = '../data/derivatives/eeg_preprocessing'
    
# iterate through participants
for sub in subjects:
    
    print(sub)
    
    for epo_type in ['stimulus', 'response']:
        print(epo_type)
    
        epochs, ar_epochs = load_epochs(sub, epo_type, layout)
        
        # visually check epochs and mark any persisting bad channels
        ar_epochs = visually_verify_epochs(epochs, ar_epochs)
        
        # average re-reference
        ar_epochs.set_eeg_reference().apply_proj()
        
        # save to file
        ar_epochs.save('%s/%s/autoreject/%s_%s_ar-epo.fif' % (pipeline_root,
                                                              sub, sub,
                                                              epo_type))
        
        # save cleaned butterfly plot
        f, ax = plt.subplots(1, 1, figsize=(20, 12))
        ar_epochs.crop(ar_epochs.times[0] + 1, ar_epochs.times[-1] - 1)
        fig = ar_epochs.average().plot(show=False, ylim=ylim, 
                                       spatial_colors=True, axes=ax)
        fig.savefig('%s/%s/%s_%s_cleaned_evoked_butterfly.png' % (pipeline_root,
                                                                  sub, sub,
                                                                  epo_type))
        plt.close('all')
        
print('Done!')

ImportError: cannot import name data